In [ ]:
# library
import numpy as np
import pandas as pd

# visualize
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

# sklearn
from sklearn import preprocessing
from sklearn import metrics
from sklearn.cluster import KMeans, MeanShift, SpectralClustering
from sklearn.decomposition import PCA, TruncatedSVD, NMF, FastICA, FactorAnalysis
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

df = pd.read_csv("data/data.csv")
submission=df[['id']]

df = df.drop(columns = "id")
cols = list(df.columns)

In [ ]:
scores = []

for sca in scalers:
  X_scaled = sca.fit_transform(df)
  X_scaled = preprocessing.PowerTransformer().fit_transform(X_scaled)
  
  X_scaled = pd.DataFrame(X_scaled, columns = cols)
  X_pca = FactorAnalysis().fit_transform(X_scaled)
  PCA_df = pd.DataFrame({"PCA_1" : X_pca[:, 0], "PCA_2" : X_pca[: , 1]})

  for gmm in autoclust:
    preds = gmm.fit_predict(X_scaled)
    PCA_df['classi'] = preds
    f, ax = plt.subplots(figsize = (5, 5))
    sns.scatterplot(data = PCA_df, x = "PCA_1", y = "PCA_2", hue = "classi"
                                   ,s = 3, palette = sns.color_palette("rocket", PCA_df["classi"].nunique()))

    plt.show()
    shs = metrics.silhouette_score(X_scaled, preds, metric = 'euclidean')
    chs=metrics.calinski_harabasz_score(X_scaled, preds)
    dbs=metrics.davies_bouldin_score(X_scaled, preds)

    scores.append([sca,gmm,shs,chs,dbs])
    print(pd.DataFrame(scores))
    submission['Predicted'] =0
    submission['Predicted']=preds
    submission.to_csv(str(gmm)+str(sca)+'submission.csv', index=False)
    submission